In [ ]:
import os
import polars as pl

# Define the data folder path
data_folder = "data"

# Get a list of all Parquet files in the data folder that do not start with 'mavir'
parquet_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith(".parquet") and not f.startswith("mavir")]

# Initialize a variable to hold the merged DataFrame
merged_df = None

# Iterate through the Parquet files and merge them sequentially
for file in parquet_files:
    df = pl.read_parquet(file)
    if merged_df is None:
        merged_df = df
    else:
        merged_df = merged_df.join(df, on="Időpont", how="inner")

# The final merged DataFrame is stored in merged_df
print(merged_df)